In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import librosa
import librosa.display

In [14]:
# Extract Raw Data Files
path = '/Users/ares/PycharmProjects/pythonProject/Emotion_Classifier/test/'
os.chdir(path)
filename = os.listdir(path)
fs = 48000

In [15]:
raw = []
length = []
label = []
for wavfile in filename:
    emotion = ''.join([i for i in wavfile[:-4]])
    #index = int(wavfile[-7:-4])
    waveform, rate = librosa.load(path + wavfile, sr = fs)
    waveform = waveform.astype('float64')
    effective = waveform[np.argwhere(waveform/max(waveform) > 0.05)[0,0]:np.argwhere(waveform/max(waveform) > 0.05)[-1,0]+1]
    raw.append(effective)
    length.append(len(effective))
    label.append(emotion)

/Users/ares/PycharmProjects/pythonProject/venv/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


NoBackendError: 

In [7]:
label = np.array(label)
raw = np.array(raw)

/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/2823102939.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw = np.array(raw)


In [10]:
# Pandas Dataframe for Features
feature = ['emotion','flatness','zerocr','meanMag','maxMag','meancent','stdcent','maxcent','stdMag'] + \
          ['pitchmean','pitchmax','pitchstd','pitch_tuning_offset','meanrms','maxrms','stdrms'] + \
          ['mfcc_' + str(i+1) + '_mean' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_std' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_max' for i in range(50)] + \
          ['chroma' + str(i+1) + '_mean' for i in range(12)] + \
          ['mels' + str(i+1) + '_mean' for i in range(128)] + \
          ['contrast' + str(i+1) + '_mean' for i in range(7)]

feature_pd = pd.DataFrame(columns=feature)

In [11]:
for i in range(315):
    saving = []
    saving.append(label[i])
    stft = np.abs(librosa.stft(raw[i]))
    #flatness
    flatness = np.mean(librosa.feature.spectral_flatness(y=raw[i]))
    saving.append(flatness)
    #zero-crossing
    zerocr = np.mean(librosa.feature.zero_crossing_rate(raw[i]))
    saving.append(zerocr)

    S, phase = librosa.magphase(stft)
    meanMagnitude = np.mean(S)
    stdMagnitude = np.std(S)
    maxMagnitude = np.max(S)
    saving.append(meanMagnitude)
    saving.append(maxMagnitude)

    #Centroid
    cent = librosa.feature.spectral_centroid(y=raw[i], sr=fs)
    cent = cent / np.sum(cent)
    meancent = np.mean(cent)
    stdcent = np.std(cent)
    maxcent = np.max(cent)
    saving.append(meancent)
    saving.append(stdcent)
    saving.append(maxcent)
    saving.append(stdMagnitude)

    # pitch
    pitches, magnitudes = librosa.piptrack(y=raw[i], sr=fs, S=stft, fmin=70, fmax=400)
    pitch = []
    for j in range(magnitudes.shape[1]):
        indexs = magnitudes[:, 1].argmax()
        pitch.append(pitches[indexs, j])
    pitch_tuning_offset = librosa.pitch_tuning(pitches)
    pitchmean = np.mean(pitch)
    pitchstd = np.std(pitch)
    pitchmax = np.max(pitch)
    pitchmin = np.min(pitch)
    saving.append(pitchmean)
    saving.append(pitchmax)
    saving.append(pitchstd)
    saving.append(pitch_tuning_offset)

    #rmse
    rmse = librosa.feature.rms(S=S)[0]
    meanrms = np.mean(rmse)
    stdrms = np.std(rmse)
    maxrms = np.max(rmse)
    saving.append(meanrms)
    saving.append(maxrms)
    saving.append(stdrms)

    #mfcc
    mfccs = np.mean(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    mfccsstd = np.std(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    mfccmax = np.max(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    saving.extend(mfccs.tolist())
    saving.extend(mfccsstd.tolist())
    saving.extend(mfccmax.tolist())

    #chromagram
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=fs).T, axis=0)
    saving.extend(chroma.tolist())

    #mels
    mel = np.mean(librosa.feature.melspectrogram(y=raw[i], sr=fs).T, axis=0)
    saving.extend(mel.tolist())
    #ottava对比
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=fs).T, axis=0)
    saving.extend(contrast.tolist())
    print(i)
    feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)

0
1
2


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


3
4
5


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


6
7
8
9


...

/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


310
311
312


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


313
314


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_12410/12768543.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


In [13]:
# Save as CSV File
feature_pd.to_csv('testcaca.csv', index=False)

In [12]:
feature_pd

,emotion,flatness,zerocr,meanMag,maxMag,meancent,stdcent,maxcent,stdMag,pitchmean,...,mels126_mean,mels127_mean,mels128_mean,contrast1_mean,contrast2_mean,contrast3_mean,contrast4_mean,contrast5_mean,contrast6_mean,contrast7_mean
0,sample126,0.015528,0.087085,0.070772,15.75928,0.00625,0.005655,0.018733,0.361107,82.917546,...,0.0,0.0,0.0,17.356484,13.458761,16.395743,16.597768,18.669926,22.138548,25.247338
1,sample132,0.009618,0.071732,0.178309,22.532114,0.008403,0.006198,0.026034,0.724208,59.667287,...,0.00001,0.000001,0.0,19.928806,13.987185,16.997238,17.294785,17.482041,23.376291,27.859975
2,sample293,0.012239,0.077164,0.315822,96.385171,0.006211,0.003992,0.017512,1.761209,67.423299,...,0.000016,0.000001,0.0,18.581371,19.169468,20.179739,19.424581,19.973051,19.451462,31.962849
3,sample287,0.009297,0.079915,0.468824,70.299249,0.007407,0.004992,0.022235,2.371914,71.206999,...,0.000052,0.000005,0.0,19.430849,15.088321,18.531389,17.738966,18.560208,24.146823,32.266224
4,sample278,0.019939,0.062083,0.036468,11.851839,0.004032,0.002874,0.018908,0.224354,101.459327,...,0.000001,0.0,0.0,20.067168,10.245237,14.382245,14.172796,17.777726,19.170206,22.080153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,sample261,0.003577,0.047503,0.094719,30.975827,0.006369,0.003669,0.019176,0.587049,121.229031,...,0.000001,0.0,0.0,17.961796,13.84973,17.6787,16.690567,19.884503,19.65532,25.197853
311,sample275,0.005997,0.049146,0.067331,15.693934,0.006024,0.003276,0.014909,0.379183,40.663742,...,0.0,0.0,0.0,18.822102,15.729889,19.049431,17.858672,20.103282,18.359068,26.559382
312,sample301,0.012417,0.066659,0.02594,8.286562,0.005814,0.003998,0.016155,0.164348,0.0,...,0.0,0.0,0.0,15.752075,14.856104,16.306742,16.804416,18.454394,18.025632,22.880373
313,sample117,0.013496,0.047937,0.018764,14.254542,0.003704,0.002247,0.01001,0.188882,2.956083,...,0.0,0.0,0.0,14.954121,15.055287,14.809998,16.169041,19.172355,18.726308,19.920733
